Graph-Based Spatio-Temporal Crime Forecasting

In [ ]:
import pandas as pd

#Load dataset (assuming it's uploaded to Colab or from your GitHub)

df = pd.read_csv('/content/drive/MyDrive/HighPerformanceMachineLearning/CrimeDatafrom2020toPresent.csv')

# Filter and preprocess (example: remove missing values, focus on top crime types)
df = df.dropna(subset=['LAT', 'LON', 'DATE OCC', 'Crm Cd Desc'])
df['DATE OCC'] = pd.to_datetime(df['DATE OCC'])

# Select top N crime types and filter
top_crimes = df['Crm Cd Desc'].value_counts().nlargest(5).index
df = df[df['Crm Cd Desc'].isin(top_crimes)]

In [ ]:
#Create weekly crime counts per location
df['Week'] = df['DATE OCC'].dt.to_period('W').apply(lambda r: r.start_time)
grouped = df.groupby(['AREA NAME', 'Week', 'Crm Cd Desc']).size().unstack(fill_value=0)

#Pivot to shape: (area, week, crime_types)
X = grouped.groupby('AREA NAME').apply(lambda x: x.reset_index(drop=True)).unstack().fillna(0)

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import kneighbors_graph

# Simulate time-series crime data for 50 areas
# Replace this with your own crime data from Step 3
num_nodes = 50
crime_types = 5
time_steps = 12
np.random.seed(42)
node_features = [np.random.rand(crime_types, time_steps) for _ in range(num_nodes)]

# Step 1: Normalize across time dimension
X_np = np.stack(node_features)  # (N, F, T)
X_np = np.nan_to_num(X_np)

scaler = StandardScaler()
flat = X_np.reshape(-1, time_steps)
X_scaled = scaler.fit_transform(flat).reshape(X_np.shape)
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)  # (N, F, T)

# Step 2: Create spatial graph using fake lat/lon (replace with real centroids)
coords = np.random.rand(num_nodes, 2) * 100
adj = kneighbors_graph(coords, n_neighbors=5, mode='connectivity', include_self=False)
adj_matrix = torch.tensor(adj.toarray(), dtype=torch.float32)

# Step 3: Add self-loops
adj_matrix += torch.eye(num_nodes)

# Step 4: Normalize adjacency matrix (GCN-style)
deg = torch.sum(adj_matrix, dim=1)
deg_inv_sqrt = torch.pow(deg, -0.5)
deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0.0
D_inv_sqrt = torch.diag(deg_inv_sqrt)
adj_norm = D_inv_sqrt @ adj_matrix @ D_inv_sqrt  # (N, N)

# Output shapes
print("X_tensor shape (node features):", X_tensor.shape)
print("adj_norm shape (normalized adjacency):", adj_norm.shape)



X_tensor shape (node features): torch.Size([50, 5, 12])
adj_norm shape (normalized adjacency): torch.Size([50, 50])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# TCN Layer
class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, dilation=1):
        super().__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size,
                              padding=(kernel_size - 1) * dilation,
                              dilation=dilation)
        self.norm = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):  # x: (N, F, T)
        x = self.conv(x)
        x = self.norm(x)
        return self.relu(x)

# GAT + TCN Hybrid Model
class CrimeGATModel(nn.Module):
    def __init__(self, in_channels, tcn_channels, out_channels):
        super().__init__()
        self.tcn = nn.Sequential(
            TemporalBlock(in_channels, tcn_channels, dilation=1),
            TemporalBlock(tcn_channels, tcn_channels, dilation=2),
        )
        self.fc = nn.Linear(tcn_channels, out_channels)

    def forward(self, x, adj):  # x: (N, F, T), adj: (N, N)
        x = self.tcn(x)                 # (N, C, T)
        x = torch.mean(x, dim=2)        # Temporal pooling → (N, C)
        x = adj @ x                     # Graph convolution via adjacency
        return self.fc(x)               # Final prediction


In [ ]:
# Setup
in_channels = X_tensor.shape[1]        # e.g., 5 crime types
tcn_channels = 64
out_channels = 1                       # Regression: crime score

# Model
model = CrimeGATModel(in_channels, tcn_channels, out_channels)

# 🔁 Enable DataParallel if multiple GPUs are available
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs with DataParallel!")
    model = nn.DataParallel(model)

# Move model to GPU
model = model.to("cuda")


In [ ]:
# Target: e.g., crime activity in last timestep
target = torch.sum(X_tensor[:, :, -1], dim=1).unsqueeze(1).to("cuda")

# Optimizer & Loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

# Train
for epoch in range(20):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    output = model(X_tensor, adj_norm)
    loss = loss_fn(output, target)

    # Backprop + optimize
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor